In [1]:
import tensorflow as tf
from tensorflow import keras 
from keras import layers,optimizers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
from keras.applications import MobileNetV2,ResNet50
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus :
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
BACH_SIZE=4
N_EPOCHS=500
LR=1e-4
PATH="D:\\df\\ai\\obd\\hand sign dataset"
columns=["path","width","height","label","xmin","ymin","xmax","ymax"]
LABELS=['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
IMAGE_WIDTH=256
IMAGE_HEIGHT=256
N_CLASSES=len(LABELS)
CSV_PATH="history.csv"
MODEL_SAVE="D:\\df\\ai\\models\\asl_obj"
S=7

In [3]:
#load data paths
def load_data(path,classes,train=True):
    images=[]
    bboxes=[]
    labels=[]
    

    if train :
        df=pd.read_csv(os.path.join(path,"train_labels.csv"),names=columns,header=1)
    else :
        df=pd.read_csv(os.path.join(path,"test_labels.csv"),names=columns,header=1)

    for _ , row in df.iterrows():
        label=row["label"]
        image=os.path.join(path,"images",row["path"])
        
        images.append(image)
        bboxes.append([int(row["xmin"]),int(row["ymin"]),int(row["xmax"]),int(row["ymax"])])
        labels.append(LABELS.index(label))

    return images,bboxes,labels

#split_dataset
def load_dataset(path,classes):
    train_images,train_bboxes,train_labels=load_data(path,classes)
    images,bboxes,labels=load_data(path,classes,False)

    test_images,val_images=train_test_split(images,test_size=0.5,random_state=18,shuffle=True)
    test_bboxes,val_bboxes=train_test_split(bboxes,test_size=0.5,random_state=18,shuffle=True)
    test_labels,val_labels=train_test_split(labels,test_size=0.5,random_state=18,shuffle=True)


    return (train_images,train_bboxes,train_labels),(val_images,val_bboxes,val_labels),(test_images,test_bboxes,test_labels)

#load images as numpy data
def read_images(image_path,bbox,label):
    image=tf.io.read_file(image_path)
    image=tf.io.decode_jpeg(image)
    height,width,channels=image.shape
    image=tf.image.resize(image,(IMAGE_HEIGHT,IMAGE_WIDTH),tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    if channels==1:
        image=tf.image.grayscale_to_rgb(image)
    image=np.array(image,dtype=np.float32)
    image=2*(image/255)-1
    
    target=np.zeros((S,S,5+N_CLASSES),dtype=np.float32)
    
    # bbox preprocess
    xmin,ymin,xmax,ymax=bbox
    xmin=xmin/width
    xmax=xmax/width
    ymin=ymin/height
    ymax=ymax/height

    x_center=(xmax+xmin)/2
    y_center=(xmax+xmin)/2

    box_height=xmax-xmin
    box_width=ymax-ymin

    #determin wich cell bbox is

    i,j=int(y_center*S),int(x_center*S)


    target[i,j,:4]=[x_center,y_center,box_width,box_height]
    target[i,j,4]=1
    target[i,j,5+label]=1

    
    
    

    return image,target

#load dataset as tensors
def load_tensors(images,bboxes,labels):
    images,labels=tf.numpy_function(read_images,[images,bboxes,labels],[tf.float32,tf.float32])
    return images,labels

def tfData(images,bbox,labels,test=False):
    ds=tf.data.Dataset.from_tensor_slices((images,bbox,labels))
    ds=ds.map(load_tensors,num_parallel_calls=tf.data.AUTOTUNE)
    ds=ds.cache()
    if not test:
        ds=ds.batch(BACH_SIZE)
    ds=ds.prefetch(tf.data.AUTOTUNE)
    return ds


(train_images,train_bboxes,train_labels),(val_images,val_bboxes,val_labels),(test_images,test_bboxes,test_labels)=load_dataset(PATH,LABELS)

ds_train=tfData(train_images,train_bboxes,train_labels)
ds_val=tfData(val_images,val_bboxes,val_labels)
print(f"train     : {len(train_images)}")
print(f"validatin : {len(val_images)}")
print(f"test      : {len(test_images)}")

train     : 1511
validatin : 36
test      : 35


In [4]:
ds=ds_train.take(3)
for i,l in ds:
    print(i.shape)

(4, 256, 256, 3)
(4, 256, 256, 3)
(4, 256, 256, 3)


In [5]:
class YoloActivation(tf.keras.layers.Layer) :
    
    def call(self, inputs) :
        coordinates = tf.nn.sigmoid(inputs[..., 0:5])
        classes = tf.nn.softmax(inputs[...,5:], axis = -1)
        return tf.concat([classes, coordinates], axis = -1)
def asl_model():
    
    input_shape=(IMAGE_HEIGHT,IMAGE_WIDTH,3)
    inputs = layers.Input(input_shape)

    pretrain = MobileNetV2(include_top=False,weights="imagenet",input_tensor=inputs)
    pretrain.trainable = True
    
    output_noeds = S*S*(5 + N_CLASSES)

    
    x = pretrain.output
    x=layers.MaxPooling2D()(x)

    x=layers.Flatten()(x)
    x=layers.Dense(512)(x)
    x=layers.LeakyReLU(0.1)(x)
    x=layers.Dropout(0.5)(x)
    x=layers.Dense(output_noeds)(x)
    x=layers.Reshape((S,S,5+ N_CLASSES))(x)
    output_layer=YoloActivation()(x)

    
    model = tf.keras.models.Model(inputs=[inputs], outputs=[output_layer])
    return model
    

model=asl_model()
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 128, 128, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [6]:
def CoordLoss(y_true, y_pred) :
    
    existsObject = tf.expand_dims(y_true[..., 4], -1)
    
    xy_pred = existsObject * y_pred[..., 0:2]
    xy_true = existsObject * y_true[..., 0:2]
    
    wh_pred = existsObject * tf.math.sign(y_pred[..., 2:4]) * tf.sqrt(tf.math.abs(y_pred[..., 2:4]))
    wh_true = existsObject * tf.sqrt(y_true[..., 2:4])
    
    coordLoss = tf.reduce_sum(tf.math.square(wh_pred - wh_true))
    coordLoss += tf.reduce_sum(tf.math.square(xy_pred - xy_true))
    
    return coordLoss / tf.cast(tf.math.count_nonzero(existsObject) , dtype = tf.float32) #for mean

def ConfidenceLoss(y_true, y_pred):
    #find if the object exists in the grid
    existsObject = tf.expand_dims(y_true[..., 4], -1)
    
    
    confidenceLoss = tf.reduce_sum(tf.math.square(existsObject * (y_true[..., 4:5] - y_pred[..., 4:5])))
    
    
    confidenceLoss += 0.5 * tf.reduce_sum(tf.math.square((1 - existsObject) * (y_true[..., 4:5] - y_pred[..., 4:5])))
    
    return confidenceLoss / tf.cast(tf.math.count_nonzero(existsObject) , dtype = tf.float32) #for mean

def ClassLoss(y_true, y_pred) :
    
    existsObject = tf.expand_dims(y_true[..., 4], -1)
    
    classLoss = tf.reduce_sum(tf.math.square(existsObject * (y_true[...,5:] - y_pred[...,5:])))
    
    return classLoss / tf.cast(tf.math.count_nonzero(existsObject) , dtype = tf.float32) #for mean
def yoloLoss(y_true, y_pred) :
    coordLoss = CoordLoss(y_true, y_pred)
    confidenceLoss = ConfidenceLoss(y_true, y_pred)
    classLoss = ClassLoss(y_true, y_pred)
    
    return 5 * coordLoss + 2 * confidenceLoss + 0.5 * classLoss

In [7]:
ds,ds2=ds_train.take(2)

print(yoloLoss(ds[1],ds2[1]))

tf.Tensor(4.531311, shape=(), dtype=float32)


In [8]:
model.compile(loss=yoloLoss ,optimizer=optimizers.Adam(LR), metrics = [CoordLoss, ConfidenceLoss, ClassLoss])

# l,pred=np.random.rand(64,S,S,N_CLASSES+5),np.random.rand(64,S,S,N_CLASSES+5)


In [9]:
from keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard


callbacks = [
        ModelCheckpoint("D:\\df\\ai\\models\\yolo", verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False),
        CSVLogger("history.csv",)
    ]

history=model.fit(
        ds_train,
        epochs=N_EPOCHS,
        validation_data=ds_val,
        verbose=1,
        callbacks=callbacks
    )

Epoch 1/500
378/378 [==============================] - ETA: 0s - loss: 5.7261 - CoordLoss: 0.6734 - ConfidenceLoss: 0.8949 - ClassLoss: 1.1372
Epoch 1: val_loss improved from inf to 4.09268, saving model to D:\df\ai\models\yolo


INFO:tensorflow:Assets written to: D:\df\ai\models\yolo\assets


INFO:tensorflow:Assets written to: D:\df\ai\models\yolo\assets


378/378 [==============================] - 59s 131ms/step - loss: 5.7261 - CoordLoss: 0.6734 - ConfidenceLoss: 0.8949 - ClassLoss: 1.1372 - val_loss: 4.0927 - val_CoordLoss: 0.3613 - val_ConfidenceLoss: 0.8932 - val_ClassLoss: 0.9996 - lr: 1.0000e-04
Epoch 2/500
378/378 [==============================] - ETA: 0s - loss: 4.1881 - CoordLoss: 0.4241 - ConfidenceLoss: 0.7806 - ClassLoss: 1.0129
Epoch 2: val_loss improved from 4.09268 to 3.45157, saving model to D:\df\ai\models\yolo


INFO:tensorflow:Assets written to: D:\df\ai\models\yolo\assets


...

INFO:tensorflow:Assets written to: D:\df\ai\models\yolo\assets


378/378 [==============================] - 45s 120ms/step - loss: 3.5854 - CoordLoss: 0.3712 - ConfidenceLoss: 0.6522 - ClassLoss: 0.8504 - val_loss: 3.2776 - val_CoordLoss: 0.2859 - val_ConfidenceLoss: 0.6872 - val_ClassLoss: 0.9473 - lr: 1.0000e-05
Epoch 21/500
378/378 [==============================] - ETA: 0s - loss: 3.5828 - CoordLoss: 0.3706 - ConfidenceLoss: 0.6530 - ClassLoss: 0.8482
Epoch 21: val_loss did not improve from 3.27759
378/378 [==============================] - 28s 74ms/step - loss: 3.5828 - CoordLoss: 0.3706 - ConfidenceLoss: 0.6530 - ClassLoss: 0.8482 - val_loss: 3.2800 - val_CoordLoss: 0.2836 - val_ConfidenceLoss: 0.6944 - val_ClassLoss: 0.9463 - lr: 1.0000e-05
Epoch 22/500
378/378 [==============================] - ETA: 0s - loss: 3.5795 - CoordLoss: 0.3703 - ConfidenceLoss: 0.6527 - ClassLoss: 0.8453
Epoch 22: val_loss improved from 3.27759 to 3.27700, saving model to D:\df\ai\models\yolo


INFO:tensorflow:Assets written to: D:\df\ai\models\yolo\assets


INFO:tensorflow:Assets written to: D:\df\ai\models\yolo\assets


378/378 [==============================] - 45s 118ms/step - loss: 3.5795 - CoordLoss: 0.3703 - ConfidenceLoss: 0.6527 - ClassLoss: 0.8453 - val_loss: 3.2770 - val_CoordLoss: 0.2848 - val_ConfidenceLoss: 0.6898 - val_ClassLoss: 0.9467 - lr: 1.0000e-05
Epoch 23/500
378/378 [==============================] - ETA: 0s - loss: 3.5766 - CoordLoss: 0.3715 - ConfidenceLoss: 0.6492 - ClassLoss: 0.8416
Epoch 23: val_loss did not improve from 3.27700
378/378 [==============================] - 28s 72ms/step - loss: 3.5766 - CoordLoss: 0.3715 - ConfidenceLoss: 0.6492 - ClassLoss: 0.8416 - val_loss: 3.2813 - val_CoordLoss: 0.2833 - val_ConfidenceLoss: 0.6958 - val_ClassLoss: 0.9465 - lr: 1.0000e-05
Epoch 24/500
378/378 [==============================] - ETA: 0s - loss: 3.5749 - CoordLoss: 0.3710 - ConfidenceLoss: 0.6502 - ClassLoss: 0.8392
Epoch 24: val_loss did not improve from 3.27700
378/378 [==============================] - 28s 73ms/step - loss: 3.5749 - CoordLoss: 0.3710 - ConfidenceLoss: 0.650